In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train_path=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
train_df=pd.DataFrame(train_path)

In [3]:
train_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [4]:
train_df.interpolate(method ='linear', limit_direction ='forward', inplace=True)
train_df.fillna(method='bfill',inplace=True)
train_df.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [5]:
train_df[['DeckName','DeckNumber','DeckSide']]=train_df['Cabin'].str.split('/',expand=True)
train_df[['Group','ID']]=train_df['PassengerId'].str.split('_',expand=True)
train_df['HomePlanet']=train_df['HomePlanet'].map({'Europa':1,'Earth':2,'Mars':3}).astype('int8')
train_df['DeckSide']=train_df['DeckSide'].map({'P':1,'S':2}).astype('int8')
train_df['DeckName']=train_df['DeckName'].map({'B':1, 'F':2, 'A':3, 'G':4, 'E':5, 'D':6, 'C':7, 'T':8}).astype('int8')
train_df['Destination']=train_df['Destination'].map({'TRAPPIST-1e':1, 'PSO J318.5-22':2, '55 Cancri e':3}).astype('int8')
train_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,DeckName,DeckNumber,DeckSide,Group,ID
0,0001_01,1,False,B/0/P,1,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,1,0,1,0001,01
1,0002_01,2,False,F/0/S,1,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,2,0,2,0002,01
2,0003_01,1,False,A/0/S,1,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,3,0,2,0003,01
3,0003_02,1,False,A/0/S,1,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,3,0,2,0003,02
4,0004_01,2,False,F/1/S,1,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,2,1,2,0004,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,1,False,A/98/P,3,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,3,98,1,9276,01
8689,9278_01,2,True,G/1499/S,2,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,4,1499,2,9278,01
8690,9279_01,2,False,G/1500/S,1,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,4,1500,2,9279,01
8691,9280_01,1,False,E/608/S,3,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,5,608,2,9280,01


In [6]:
train_df['CryoSleep']=train_df['CryoSleep'].astype(int)
train_df['VIP']=train_df["VIP"].astype(int)
train_df['Transported']=train_df['Transported'].astype(int)

In [7]:
train_df.head(100)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,DeckName,DeckNumber,DeckSide,Group,ID
0,0001_01,1,0,B/0/P,1,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1,0,1,0001,01
1,0002_01,2,0,F/0/S,1,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,2,0,2,0002,01
2,0003_01,1,0,A/0/S,1,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,3,0,2,0003,01
3,0003_02,1,0,A/0/S,1,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,3,0,2,0003,02
4,0004_01,2,0,F/1/S,1,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,2,1,2,0004,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0103_01,2,0,F/24/S,1,24.0,0,0.0,0.0,0.0,0.0,17.0,Arlen Mclainez,1,2,24,2,0103,01
96,0103_02,2,1,G/18/S,3,9.0,0,0.0,0.0,0.0,0.0,0.0,Shaele Mclainez,1,4,18,2,0103,02
97,0103_03,2,1,G/18/S,1,6.0,0,0.0,0.0,0.0,0.0,0.0,Fanne Mclainez,0,4,18,2,0103,03
98,0105_01,2,0,F/21/P,1,27.0,0,0.0,0.0,570.0,2.0,131.0,Carry Cleachrand,0,2,21,1,0105,01


In [8]:
x_train=train_df[['Group','HomePlanet','CryoSleep','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','DeckName','DeckNumber','DeckSide']]
y_train=train_df['Transported']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split( x_train, y_train, test_size=0.33, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestClassifier
# n_estimators = number of decision trees
rf = RandomForestClassifier(n_estimators=78, max_depth=17)
rf.fit(x_train, y_train)
print("score on test: " + str(rf.score(X_test, Y_test)))
print("score on train: "+ str(rf.score(X_train, Y_train)))

score on test: 0.9839665388637155
score on train: 0.9833447802197802


In [11]:
test_path=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [12]:
test_df=pd.DataFrame(test_path)
test_df.interpolate(method ='linear', inplace=True)
test_df.fillna(method='bfill',inplace=True)

In [13]:
test_df[['DeckName','DeckNumber','DeckSide']]=test_df['Cabin'].str.split('/',expand=True)
test_df[['Group','ID']]=test_df['PassengerId'].str.split('_',expand=True)
test_df['HomePlanet']=test_df['HomePlanet'].map({'Europa':1,'Earth':2,'Mars':3}).astype('int8')
test_df['DeckSide']=test_df['DeckSide'].map({'P':1,'S':2}).astype('int8')
test_df['DeckName']=test_df['DeckName'].map({'B':1, 'F':2, 'A':3, 'G':4, 'E':5, 'D':6, 'C':7, 'T':8}).astype('int8')
test_df['Destination']=test_df['Destination'].map({'TRAPPIST-1e':1, 'PSO J318.5-22':2, '55 Cancri e':3}).astype('int8')
test_df['CryoSleep']=test_df['CryoSleep'].astype(int)
test_df['VIP']=test_df["VIP"].astype(int)

In [14]:
test_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,DeckName,DeckNumber,DeckSide,Group,ID
0,0013_01,2,1,G/3/S,1,27.000000,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,4,3,2,0013,01
1,0018_01,2,0,F/4/S,1,19.000000,0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,2,4,2,0018,01
2,0019_01,1,1,C/0/S,3,31.000000,0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,7,0,2,0019,01
3,0021_01,1,0,C/1/S,1,38.000000,0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,7,1,2,0021,01
4,0023_01,2,0,F/5/S,1,20.000000,0,10.0,0.0,635.0,0.0,0.0,Brence Harperez,2,5,2,0023,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,2,1,G/1496/S,1,34.000000,0,0.0,0.0,0.0,0.0,0.0,Jeron Peter,4,1496,2,9266,02
4273,9269_01,2,0,D/296/P,1,42.000000,0,0.0,847.0,17.0,10.0,144.0,Matty Scheron,6,296,1,9269,01
4274,9271_01,3,1,D/296/P,3,42.333333,0,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,6,296,1,9271,01
4275,9273_01,1,0,D/297/P,2,42.666667,0,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,6,297,1,9273,01


In [15]:
x_test=test_df[['Group','HomePlanet','CryoSleep','Destination','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','DeckName','DeckNumber','DeckSide']]
passengers=test_df['PassengerId']
y_test=rf.predict(x_test)
final=pd.DataFrame(data=(passengers),columns=['PassengerId'])
final.loc[:,"Transported"]=y_test.astype('bool')
final.head(20)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
5,0027_01,False
6,0029_01,True
7,0032_01,True
8,0032_02,True
9,0033_01,True


In [16]:
final.to_csv('submission',index=False)
sub=pd.read_csv('./submission')
sub.head(25)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
5,0027_01,False
6,0029_01,True
7,0032_01,True
8,0032_02,True
9,0033_01,True
